# Darius — NILM CNN Training Loop



In [1]:
# --- Colab setup ---
!git clone https://github.com/gbauer-at-sandiego-edu/smart-home-energy-optimizer.git
%cd smart-home-energy-optimizer

!pip -q install kagglehub python-dotenv seaborn scikit-learn tqdm


Cloning into 'smart-home-energy-optimizer'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 118 (delta 50), reused 20 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (118/118), 1.08 MiB | 3.73 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/smart-home-energy-optimizer


In [2]:
# --- Imports ---
import os
import json
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


## 1) Build / load processed dataset

This uses the repo's ingest script to download UK-DALE from KaggleHub and produce a 1-minute processed CSV with a `kettle_on` label.

In [4]:
# Run the repo's ingest to create: data/processed/building1_mains_kettle_1min_180d.csv
!python -m src.data_ingest

DATA_CSV = Path('data/processed/building1_mains_kettle_1min_180d.csv')
assert DATA_CSV.exists(), f"Missing {DATA_CSV}"

df = pd.read_csv(DATA_CSV)
# First column is timestamp index written by pandas
if 'kettle_on' not in df.columns:
    raise ValueError('Expected kettle_on column in processed CSV')

# Rename timestamp column to something consistent
df = df.rename(columns={df.columns[0]: 'timestamp'})
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Basic cleanup
df = df.dropna(subset=['mains_W','kettle_W','kettle_on']).reset_index(drop=True)
df['kettle_on'] = df['kettle_on'].astype(int)

print(df.head())
print('Rows:', len(df))
print("kettle_on counts:\n", df["kettle_on"].value_counts(dropna=False))

100% 5.06G/5.06G [00:58<00:00, 93.1MB/s]
Extracting files...
H5 path: /root/.cache/kagglehub/datasets/abdelmdz/uk-dale/versions/1/ukdale.h5
Rows: 167117
                               mains_W  kettle_W  kettle_on
2012-12-14 22:21:00+00:00  1001.666687       1.0          0
2012-12-14 22:22:00+00:00   978.888916       1.0          0
2012-12-14 22:23:00+00:00   992.555542       1.0          0
2012-12-14 22:24:00+00:00  1015.900024       1.0          0
2012-12-14 22:25:00+00:00  1004.900024       1.0          0
kettle_on counts:
kettle_on
0    165950
1      1167
Name: count, dtype: int64
Wrote: data/processed/building1_mains_kettle_1min_180d.csv
                   timestamp     mains_W  kettle_W  kettle_on
0  2012-12-14 22:21:00+00:00  1001.66670       1.0          0
1  2012-12-14 22:22:00+00:00   978.88890       1.0          0
2  2012-12-14 22:23:00+00:00   992.55554       1.0          0
3  2012-12-14 22:24:00+00:00  1015.90000       1.0          0
4  2012-12-14 22:25:00+00:00  1004.90000

/tmp/ipython-input-1822290706.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')


## 2) Windowing + normalization

We create fixed-length windows from `mains_W` and label each window by whether a kettle ON event occurs at the **end** of the window (you can adjust labeling logic).

In [5]:
# Hyperparameters (adjust as needed)
WINDOW = 256          # samples (1-min resolution -> 256 minutes)
STRIDE = 32           # step size
TEST_SIZE = 0.15
VAL_SIZE = 0.15       # fraction of remaining train
SEED = 42

mains = df['mains_W'].astype('float32').to_numpy()
y_raw = df['kettle_on'].astype('int32').to_numpy()

# Robust normalization using training-only stats later (we compute after split indices)

# Build windows
X = []
y = []
end_idxs = []
for end in range(WINDOW, len(mains), STRIDE):
    start = end - WINDOW
    xw = mains[start:end]
    label = y_raw[end-1]  # label at window end
    X.append(xw)
    y.append(label)
    end_idxs.append(end-1)

X = np.array(X, dtype='float32')
y = np.array(y, dtype='int32')

# Add channel dim for CNN: (N, T, C)
X = X[..., np.newaxis]

print('X shape:', X.shape, 'y shape:', y.shape)
print('Positive rate:', y.mean())


X shape: (5215, 256, 1) y shape: (5215,)
Positive rate: 0.007670182166826462


## 3) Train/val/test split (time-safe option)

For NILM, random splits can leak temporal structure. Below is a simple **chronological** split.

In [6]:
# Chronological split
N = len(X)
idx = np.arange(N)

# test split at end
test_n = int(N * TEST_SIZE)
trainval_idx, test_idx = idx[:-test_n], idx[-test_n:]

# val split from end of trainval
val_n = int(len(trainval_idx) * VAL_SIZE)
train_idx, val_idx = trainval_idx[:-val_n], trainval_idx[-val_n:]

X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]
X_test, y_test = X[test_idx], y[test_idx]

print('Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
print('Train pos rate:', y_train.mean(), 'Val pos rate:', y_val.mean(), 'Test pos rate:', y_test.mean())


Train: (3769, 256, 1) Val: (664, 256, 1) Test: (782, 256, 1)
Train pos rate: 0.00716370390023879 Val pos rate: 0.009036144578313253 Test pos rate: 0.008951406649616368


In [7]:
# Normalize using TRAIN statistics only
train_mean = X_train.mean()
train_std = X_train.std() + 1e-6

X_train_n = (X_train - train_mean) / train_std
X_val_n   = (X_val   - train_mean) / train_std
X_test_n  = (X_test  - train_mean) / train_std

print('mean/std:', float(train_mean), float(train_std))


mean/std: 415.7292785644531 444.7979431152344


## 4) Class imbalance handling

In [8]:
# Compute simple class weights
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

# Avoid divide-by-zero
if pos == 0:
    raise ValueError('No positive examples in training split. Reduce WINDOW/STRIDE or adjust label logic.')

w0 = 0.5 * (len(y_train) / neg)
w1 = 0.5 * (len(y_train) / pos)
class_weight = {0: w0, 1: w1}
print('class_weight:', class_weight)


class_weight: {0: np.float64(0.5036076964190273), 1: np.float64(69.79629629629629)}


## 5) CNN model + training loop with logging/checkpoints

In [9]:
def make_model(window=WINDOW):
    inp = keras.Input(shape=(window, 1))
    x = layers.Conv1D(32, 7, padding='same', activation='relu')(inp)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(64, 5, padding='same', activation='relu')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D(128, 3, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inp, out)
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss='binary_crossentropy',
        metrics=[
            keras.metrics.BinaryAccuracy(name='acc'),
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc'),
        ],
    )
    return model

model = make_model()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 256, 32)        │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 128, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 128, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 64, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,585 (170.25 KB)

 Trainable params: 43,585 (170.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Outputs
Path('reports').mkdir(exist_ok=True)
Path('models').mkdir(exist_ok=True)

run_name = 'cnn_kettle_on'

callbacks = [
    keras.callbacks.CSVLogger(f'reports/{run_name}_history.csv', append=False),
    keras.callbacks.ModelCheckpoint(
        filepath=f'models/{run_name}.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1,
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=5,
        restore_best_weights=True,
        verbose=1,
    ),
]

history = model.fit(
    X_train_n, y_train,
    validation_data=(X_val_n, y_val),
    epochs=30,
    batch_size=128,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1,
)


Epoch 1/30
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - acc: 0.1351 - auc: 0.4837 - loss: 0.7780 - precision: 0.0076 - recall: 0.7674
Epoch 1: val_auc improved from -inf to 0.43541, saving model to models/cnn_kettle_on.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 143ms/step - acc: 0.1431 - auc: 0.4870 - loss: 0.7724 - precision: 0.0076 - recall: 0.7681 - val_acc: 0.5843 - val_auc: 0.4354 - val_loss: 0.6923 - val_precision: 0.0073 - val_recall: 0.3333
Epoch 2/30
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - acc: 0.6657 - auc: 0.5187 - loss: 0.6866 - precision: 0.0068 - recall: 0.3109
Epoch 2: val_auc improved from 0.43541 to 0.45555, saving model to models/cnn_kettle_on.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - acc: 0.6636 - auc: 0.5231 - loss: 0.6866 - precision: 0.0070 - recall: 0.3219 - val_acc: 0.5602 - val_auc: 0.4555 - val_loss: 0.6694 - val_precision: 0.0069 - val_recall: 0.3333
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - acc: 0.6175 - auc: 0.6721 - loss: 0.6590 - precision: 0.01

## 6) Evaluation (confusion matrix + classification report)

In [12]:
# Predict
y_prob = model.predict(X_test_n).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("Classification report:")
print(classification_report(y_test, y_pred, digits=4))

print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

# Save run config/normalization stats
metrics = {
    "window": int(WINDOW),
    "stride": int(STRIDE),
    "train_mean": float(train_mean),
    "train_std": float(train_std),
}
Path("reports").mkdir(exist_ok=True)
with open(f"reports/{run_name}_config.json", "w") as f:
    json.dump(metrics, f, indent=2)

print("\nSaved reports + model artifacts to ./reports and ./models")


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step
Classification report:
              precision    recall  f1-score   support

           0     0.9986    0.9523    0.9749       775
           1     0.1395    0.8571    0.2400         7

    accuracy                         0.9514       782
   macro avg     0.5691    0.9047    0.6075       782
weighted avg     0.9910    0.9514    0.9683       782


Confusion matrix:
[[738  37]
 [  1   6]]

Saved reports + model artifacts to ./reports and ./models
